In [2]:
import os

import numpy as np
import faiss
from tqdm import tqdm

In [3]:
index_dir = "/home/kucerj56/FlashRAG/experiments/build_index"
files = [file for file in os.listdir(index_dir)]
len(files)

17790

In [4]:
files = list(filter(lambda x: x.endswith(".npy"), files))
len(files)

17784

In [5]:
files = sorted(files, key=lambda x: int(x.split('.')[0].split('_')[-1]))

In [6]:
files[:10]

['query_emb_batch_0.npy',
 'query_emb_batch_512.npy',
 'query_emb_batch_1024.npy',
 'query_emb_batch_1536.npy',
 'query_emb_batch_2048.npy',
 'query_emb_batch_2560.npy',
 'query_emb_batch_3072.npy',
 'query_emb_batch_3584.npy',
 'query_emb_batch_4096.npy',
 'query_emb_batch_4608.npy']

In [7]:
embds = []
for file in tqdm(files):
    embds.append(np.load(os.path.join(index_dir, file)))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17784/17784 [02:47<00:00, 105.87it/s]


In [8]:
embs = embds

In [9]:
concat_embds = np.concatenate(embs, axis=0)

In [10]:
embs = embds = None
# delete(embs)
# delete(embds)

In [ ]:
# TODO L2 normalise?

In [12]:
dim = concat_embds.shape[-1]
# faiss_index = faiss.index_factory(dim, "Flat", faiss.METRIC_INNER_PRODUCT)
faiss_index = faiss.index_factory(dim, "IVF4096,PQ64x8", faiss.METRIC_INNER_PRODUCT)

In [14]:
faiss_index.is_trained

False

In [15]:
faiss_index.train(concat_embds)

In [16]:
faiss_index.add(concat_embds)

In [17]:
# faiss.write_index(faiss_index, "/home/kucerj56/datasets/index/wiki_2018_e5_base_flat.index")
faiss.write_index(faiss_index, "/home/kucerj56/datasets/index/wiki_2018_e5_base_IVF4096_PQ64x8.index")

In [21]:
memmap = np.memmap("/home/kucerj56/datasets/index/wiki_embeddings.npy", shape=concat_embds.shape, mode="w+", dtype=concat_embds.dtype)

In [23]:
length = concat_embds.shape[0]
print(f"shape of all_embeddings: {concat_embds.shape}")
# add in batch
save_batch_size = 10000
if length > save_batch_size:
    for i in tqdm(range(0, length, save_batch_size), leave=False, desc="Saving Embeddings"):
        j = min(i + save_batch_size, length)
        memmap[i:j] = concat_embds[i:j]
else:
    memmap[:] = concat_embds


shape of all_embeddings: (9105161, 768)
